In [1]:
# Module importing
import pandas as pd

#Dataset Path
DATASET_PATH = "./data/"

In [2]:
# Importing dataset

# NOTA: hay que hacer un pip install pyarrow para que funcione

df_feb= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-02.parquet', engine='pyarrow')
df_aug= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-08.parquet', engine='pyarrow')
df_nov= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-11.parquet', engine='pyarrow')

# 0- Definicion de scope del trabajo

## 0.1 -Preguntas a responder 

Primera pregunta:
- **Analsis de datos**: ¿Existe una manera de caracterizar los lugares más recurrentes para inicio/fin de viaje?<br>
- **Aprendizaje de maquina**: buscar si existen Lugares óptimos para recoger pasajeros basados en la info que tienen (el punto de optimización sería a decisión de ustedes, si fare_amount o tip_amount u otra variable)<br><br>

Segunda pregunta:
- **Analsis de datos**: Como se comportó la propina de los viajes?  ¿Podremos segmentar los viajes de acuerdo a la propina? (clusterización)<br>
- **Aprendizaje de maquina**: Crear una variable booleana que contemple por ejemplo si el viaje dado tiene una propina mayor a algun valor (por ejemplo la media podada, o la moda) y así poder clasificar el viaje entre buena o mala propina.



## 0.2 Decisiones de diseño y pre-procesamiento

- Se resuelve dropear muestras que tengan valores negativos en la propina y/o total amount. Estos valores pueden corresponder a errores en el dataset o reversiones de pago. 

# 1- Analisis exploratorio inicial

## 1.1 - Dataset features and characteristics

In [3]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-02-01 00:17:35,2020-02-01 00:30:32,1.0,2.60,1.0,N,145,7,1,11.0,0.5,0.5,2.45,0.0,0.3,14.75,0.0,None
1,1,2020-02-01 00:32:47,2020-02-01 01:05:36,1.0,4.80,1.0,N,45,61,1,21.5,3.0,0.5,6.30,0.0,0.3,31.60,2.5,None
2,1,2020-02-01 00:31:44,2020-02-01 00:43:28,1.0,3.20,1.0,N,186,140,1,11.0,3.0,0.5,1.00,0.0,0.3,15.80,2.5,None
3,2,2020-02-01 00:07:35,2020-02-01 00:31:39,1.0,4.38,1.0,N,144,140,1,18.0,0.5,0.5,3.00,0.0,0.3,24.80,2.5,None
4,2,2020-02-01 00:51:43,2020-02-01 01:01:29,1.0,2.28,1.0,N,238,152,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,0.0,None


In [4]:
df_feb.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
6299362,2,2020-02-29 23:24:00,2020-02-29 23:55:00,NaN,10.59,NaN,None,225,205,0,48.85,2.75,0.5,0.0,0.00,0.3,52.40,NaN,None
6299363,6,2020-02-29 23:02:01,2020-02-29 23:02:25,NaN,5.88,NaN,None,265,226,0,26.65,2.33,0.5,0.0,0.00,0.3,32.28,NaN,None
6299364,2,2020-02-29 23:35:00,2020-02-29 23:58:00,NaN,4.46,NaN,None,45,80,0,24.70,2.75,0.5,0.0,0.00,0.3,28.25,NaN,None
6299365,2,2020-02-29 23:42:00,2020-03-01 00:11:00,NaN,15.73,NaN,None,235,197,0,53.03,2.75,0.5,0.0,6.12,0.3,62.70,NaN,None
6299366,6,2020-02-29 23:02:13,2020-03-01 00:03:24,NaN,11.34,NaN,None,265,116,0,37.62,3.29,0.5,0.0,0.00,0.3,44.21,NaN,None


In [5]:
df_feb.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,6.299367e+06,6.250533e+06,6.299367e+06,6.250533e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.299367e+06,6.250533e+06
mean,1.673616e+00,1.504449e+00,2.864533e+00,1.055871e+00,1.646020e+02,1.625341e+02,1.248365e+00,1.260736e+01,1.108678e+00,4.927634e-01,2.220724e+00,3.273723e-01,2.979165e-01,1.859116e+01,2.307304e+00
std,4.692367e-01,1.144727e+00,4.027472e+01,7.769877e-01,6.554007e+01,6.977452e+01,4.793241e-01,1.189053e+01,1.255356e+00,7.366464e-02,2.665063e+00,1.675211e+00,3.452216e-02,1.445150e+01,6.903989e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-7.500000e+02,-1.769000e+01,-5.000000e-01,-9.300000e+01,-3.823000e+01,-3.000000e-01,-7.503000e+02,-2.500000e+00
25%,1.000000e+00,1.000000e+00,9.600000e-01,1.000000e+00,1.250000e+02,1.130000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.600000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.960000e+00,0.000000e+00,3.000000e-01,1.452000e+01,2.500000e+00
75%,2.000000e+00,2.000000e+00,2.900000e+00,1.000000e+00,2.330000e+02,2.340000e+02,1.000000e+00,1.400000e+01,2.500000e+00,5.000000e-01,2.950000e+00,0.000000e+00,3.000000e-01,2.015000e+01,2.500000e+00
max,6.000000e+00,9.000000e+00,5.705109e+04,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,6.052000e+03,9.006000e+01,3.951000e+01,5.490200e+02,9.255000e+02,3.000000e-01,6.061420e+03,2.750000e+00


In [6]:
df_feb.median()

C:\Users\Esgoty\AppData\Local\Temp\ipykernel_10160\933812902.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_feb.median()
C:\Users\Esgoty\AppData\Local\Temp\ipykernel_10160\933812902.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_feb.median()


VendorID                   2.00
passenger_count            1.00
trip_distance              1.60
RatecodeID                 1.00
PULocationID             162.00
DOLocationID             162.00
payment_type               1.00
fare_amount                9.00
extra                      0.50
mta_tax                    0.50
tip_amount                 1.96
tolls_amount               0.00
improvement_surcharge      0.30
total_amount              14.52
congestion_surcharge       2.50
airport_fee                 NaN
dtype: float64

## 1.2 Tipos de datos y tipos de variables

In [7]:
df_feb.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                      object
dtype: object

## 1.2.1 Tipos de variables del dataset

Se analiza la descripcion de las features del dataset provistas por el proveedor del dataset. 
File: data_dictionary_trip_records_yellow.pdf

### Numericas Discretas (variable no puede tomar valores entre dos valores contiguos)
passenger_count --------------------- float64  //los pasajeros saltan de a 1 <br>
tolls_amount --------------------- float64 <br>
% es funcion de variables discretas es decir cobinacion lineal de 0,1,2,3 etc peajes que hubo durante el viaje.  <br>
extra --------------------- float64 --------------------- Cardinalidad:3 // tres categorias disitntas<br>  <br> 

NOTA: Son informativas para problemas de clasificacion. 


### Numericas Continuas (variable puede tomar valores en un rango)
trip_distance --------------------- float64<br>
fare_amount --------------------- float64<br> 
tip_amount --------------------- loat64<br>
total_amount --------------------- float64<br>

### Categoricas Nominales (variable qualitativa, pone un nombre/label sin orden interinseco entre valores)
VendorID --------------------- int64 --------------------- Cardinalidad:2 // nombre de empresa<br>
congestion_surcharge --------------------- float64 --------------------- Cardinalidad:2 // se cobra o no<br>
payment_type --------------------- int64 --------------------- Cardinalidad:6 // forma de pago<br> 
airport_fee  --------------------- object --------------------- Cardinalidad:2 // se cobra o no<br>
mta_tax --------------------- float64 --------------------- Cardinalidad:2 // se cobra o no <br>
improvement_surcharge  --------------------- float64 --------------------- Cardinalidad:2 //se cobra o no <br>
store_and_fwd_flag --------------------- object --------------------- Cardinalidad:2 // dos categorias <br>
RatecodeID --------------------- float64 --------------------- Cardinalidad:6 // ID de la tarifa seleccionada al momento de terminar el viaje <br>

NOTA: Son informativas para problemas de clasificacion. 


### categoricas ordinales (variable qualitativa, pone un nombre/label CON orden interinseco entre valores)
PULocationID --------------------- int64 --------------------- Cardinalidad:muchas // nombre, orden es localizaion geografica<br> 
DOLocationID --------------------- int64 --------------------- Cardinalidad:muchas // nombre, orden es localizaion geografica<br>

NOTA: Son informativas para problemas de clasificacion. 



### Fecha y hora
tpep_pickup_datetime --------------------- datetime64[ns]<br>
tpep_dropoff_datetime --------------------- datetime64[ns]<br>

### Compuestas
//no hay

## 1.2.2 Identificacion de variables de entrada y de salida

### Dataset y definiciones iniciales:  
**Variables de entrada:**<br>
    Todas las features del dataset. <br>
**Variables de salida:** <br>
    El dataset y la defincion incial del problema no definen una variable de salida y/o label. <br>


### Primera pregunta: 
**Analsis de datos**: ¿Existe una manera de caracterizar los lugares más recurrentes para inicio/fin de viaje?<br>
**Aprendizaje de maquina**: buscar si existen Lugares óptimos para recoger pasajeros basados en la info que tienen (el punto de optimización sería a decisión de ustedes, si fare_amount o tip_amount u otra variable)<br><br>


**Variables de entrada:**<br>
    Todas las features del dataset exeptuando las que se definieron como salida para la pregunta 1. <br>
**Variables de salida:** <br>
    PULocationID: es ID de area en donde comienza el viaje <br>
    DOLocationID: es ID de area en donde finaliza el viaje <br>




### Segunda pregunta: 
**Analsis de datos**: Como se comportó la propina de los viajes?  ¿Podremos segmentar los viajes de acuerdo a la propina? (clusterización)<br>
**Aprendizaje de maquina**: Crear una variable booleana que contemple por ejemplo si el viaje dado tiene una propina mayor a algun valor (por ejemplo la media podada, o la moda) y así poder clasificar el viaje entre buena o mala propina.

**Variables de entrada:**<br>
    Todas las features del dataset exeptuando las que se definieron como salida para la pregunta 2. <br>
**Variables de salida:** <br>
    Nueva variable Boleana que clasifica si la propina es buena o mala. (Umbral entre buena y mala propina US$ 2)



## 1.2.3 Estudio de distribuciones y caracterisitcas de features del dataset.


0- Verificacion de ocurrencia de valores faltantes
Se comprueba la presencia de valores faltantes de los siguientes tipos:
- NaNs  (por ej en feature tail congestion_surcharge)
- None  (por ej en feature tail airport_fee)


1- Analisis de valores distintos para cada variable.
Del analisis podemos ver que airport fee es una feature que no toma ningun valor. Y se puede dropear del todo. 

Explicación: Debido a que el dataset es de febrero de 2020 la ausencia de valores en airportfee puede explicarse a la prohibicion de tomar viajes aereos vigente en ese momento. <br><br><br>

2- analisis de valores anomalos dentro de una feature
Se realizo analisis de maximos y minimos para cada feature, y se comprobo la presencia de valores extremos anomalos que teniendo en cuenta la descripcion de features no tienen sentido en el contexto del dataset. Estos valores pueden corresponder a errores en el dataset o reversiones de pago. <br>
Por ejemplo, la variable de total amount (cantidad de dinero pagada por un cliente) tiene un valor minimo de -750 <br>




**NOTA 1:**
Durante el analisis inicial, se comprueba la presencia de valores faltantes o NaNs. Por lo cual el presente analisis se halla restringido a un analisis exploratorio dado que no se han gestionado varlores faltantes, ni realizado imputaciones de los mismos. Lo cual actua en detrimento de las herramientas de la estadisitca descriptiva para las variables numericas. <br><br>

**NOTA 2:**
En analisis posteriores se realizara la modificacion del dataset para ajustarlo a las caracteristicas esperadas por el modelo de ML seleccionado en cada caso.   <br><br>

In [15]:
df_feb.isna().values.any()

True

In [13]:
df_feb.nunique()

VendorID                       4
tpep_pickup_datetime     2027136
tpep_dropoff_datetime    2031299
passenger_count               10
trip_distance               4391
RatecodeID                     7
store_and_fwd_flag             2
PULocationID                 259
DOLocationID                 261
payment_type                   5
fare_amount                 5137
extra                        122
mta_tax                       11
tip_amount                  3680
tolls_amount                 937
improvement_surcharge          3
total_amount               12311
congestion_surcharge           7
airport_fee                    0
dtype: int64

## 1.3 Handling missing data

In [11]:
pd.isna(df_feb)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299362,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
6299363,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
6299364,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
6299365,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True


In [12]:
df_feb.isnull().sum()

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count            48834
trip_distance                  0
RatecodeID                 48834
store_and_fwd_flag         48834
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge       48834
airport_fee              6299367
dtype: int64

A 1 feature
2a- Obtener metricas estadisticas para las numericas (clasificar distribuciones, ratings de las distribcuiones(mediana, soporte,media,otros momentos como la curtosis, etc) )

2a- Obtener metricas estadisticas para las numericas (moda,Balanceado,valores frecuentes, soporte(cardinalidad) )

A 2 feature
2a- Obtener metricas estadisticas entre variables (solo numericas)

2b- Obtener metricas estadisticas entre variables (que se puede hacer con categoricas????????)

# Annex 1- tipos de variables
Tipos de variables 

Categóricas. Datos con un número limitado de valores o categorías distintas (por ejemplo, género o religión). Las variables categóricas pueden ser variables de cadena (alfanuméricas) o variables numéricas que utilizan códigos numéricos para representar a categorías (por ejemplo, 0 = hombre y 1 = mujer). También se hace referencia a estos datos como datos cualitativos. Las variables categóricas pueden ser nominales u ordinales

Nominal. Una variable puede tratarse como nominal cuando sus valores representan categorías sin clasificación intrínseca (por ejemplo, el departamento de la empresa en la que trabaja un empleado). Ejemplos de variables nominales incluyen la región, el código postal y la afiliación religiosa.

Ordinal. Una variable puede ser tratada como ordinal cuando sus valores representan categorías con alguna clasificación intrínseca (por ejemplo, niveles de satisfacción de servicio de altamente insatisfecho a altamente satisfecho). Ejemplos de variables ordinales incluyen puntuaciones de actitud que representan grado de satisfacción o confianza y puntuaciones de calificación de preferencia.


CATEGORICAL VARIABLES: In statistics, a categorical variable (also called qualitative variable) is a variable that can take on one of a limited, and usually fixed, number of possible values, assigning each individual or other unit of observation to a particular group or nominal category on the basis of some qualitative property

NOMINAL VARIABLES: A nominal category or a nominal group is a group of objects or ideas that can be collectively grouped on the basis of a particular characteristic—a qualitative property. A variable that codes whether each one in a set of observations is in a particular nominal category is called a categorical variable. 

ORDINAL VARIABLES: Ordinal data is a categorical, statistical data type where the variables have natural, ordered categories and the distances between the categories are not known.